In [2]:
import pandas as pd
import sqlite3

Import full census block group data, generate one row per 3 h block group per day per census line

In [3]:
df_census = pd.read_csv("DC-Criminalistics/data/census-data/FinalBlockGroupData.csv")

In [4]:
start_year = '2013'
end_year = '2017'


df_time = pd.DataFrame({
    'date': pd.date_range(
        start = pd.Timestamp(start_year),
        end = pd.Timestamp(end_year) + pd.offsets.YearEnd(0) + pd.Timedelta(days=1),
        freq = '3h',
        closed = 'left'
    )
})

Add columns for month/day/year/3-hr time block category column

In [5]:
df_time['date'] = pd.to_datetime(df_time['date'])
df_time['year'] = df_time['date'].dt.year
df_time['month'] = df_time['date'].dt.month
df_time['day'] = df_time['date'].dt.day
df_time['time_block'] = pd.cut(df_time['date'].dt.hour,8,labels=['12am-3am','3am-6am','6am-9am','9am-12pm','12pm-3pm','3pm-6pm','6pm-9pm','9pm-12am'],include_lowest=True)

In [6]:
df_time.head(10)

,date,year,month,day,time_block
0,2013-01-01 00:00:00,2013,1,1,12am-3am
1,2013-01-01 03:00:00,2013,1,1,3am-6am
2,2013-01-01 06:00:00,2013,1,1,6am-9am
3,2013-01-01 09:00:00,2013,1,1,9am-12pm
4,2013-01-01 12:00:00,2013,1,1,12pm-3pm
5,2013-01-01 15:00:00,2013,1,1,3pm-6pm
6,2013-01-01 18:00:00,2013,1,1,6pm-9pm
7,2013-01-01 21:00:00,2013,1,1,9pm-12am
8,2013-01-02 00:00:00,2013,1,2,12am-3am
9,2013-01-02 03:00:00,2013,1,2,3am-6am


Merge with census on Year

In [7]:
full_date_time_census = df_time.merge(df_census, left_on='year', right_on='Year')

In [8]:
full_date_time_census.head(10)

,date,year,month,day,time_block,Unnamed: 0,TotalPop,TPopMargin,UnWgtSampleCtPop,PerCapitaIncome,...,MedianAge,MedianAgeMargin,HousingUnits,HousingUnitsMargin,UnweightedSampleHousingUnits,State,County,Tract,BlockGroup,Year
0,2013-01-01,2013,1,1,12am-3am,433,785.0,326.0,73.0,32862.0,...,40.5,11.3,359.0,85.0,29.0,11,1,2202,3,2013
1,2013-01-01,2013,1,1,12am-3am,434,1247.0,245.0,110.0,34004.0,...,38.5,10.2,442.0,71.0,46.0,11,1,2301,1,2013
2,2013-01-01,2013,1,1,12am-3am,435,1621.0,232.0,153.0,34506.0,...,38.9,4.3,721.0,76.0,68.0,11,1,2301,2,2013
3,2013-01-01,2013,1,1,12am-3am,436,1808.0,277.0,149.0,34318.0,...,36.0,20.8,1093.0,101.0,68.0,11,1,2302,1,2013
4,2013-01-01,2013,1,1,12am-3am,437,787.0,213.0,64.0,34204.0,...,40.9,14.3,349.0,77.0,26.0,11,1,2400,1,2013
5,2013-01-01,2013,1,1,12am-3am,438,1018.0,239.0,91.0,40622.0,...,37.5,6.4,426.0,79.0,36.0,11,1,2400,2,2013
6,2013-01-01,2013,1,1,12am-3am,439,881.0,388.0,69.0,38477.0,...,31.8,10.5,303.0,73.0,23.0,11,1,2400,3,2013
7,2013-01-01,2013,1,1,12am-3am,440,1228.0,312.0,105.0,37168.0,...,37.7,3.7,352.0,75.0,34.0,11,1,2400,4,2013
8,2013-01-01,2013,1,1,12am-3am,441,1145.0,369.0,62.0,14490.0,...,30.4,10.1,520.0,83.0,34.0,11,1,2501,1,2013
9,2013-01-01,2013,1,1,12am-3am,442,1371.0,298.0,124.0,49665.0,...,40.3,5.0,549.0,85.0,46.0,11,1,2501,2,2013


In [9]:
#conn = sqlite3.connect('DC-Criminalistics/data/full_date_time_census.db')
#full_date_time_census.to_sql('full_date_time_census', conn)

Import Census/Crime/Weather to figure out how to merge

In [ ]:
conn = sqlite3.connect('DC-Criminalistics_06012019/data/crime_census_weather.db')
c = conn.cursor()
#weather_df = pd.read_sql('''select * from weather_data''', conn)
df = pd.read_sql('''select * from crime_census_weather where year >=2013 and year <= 2017''', conn)

In [ ]:
df.sort_values('Tract', ascending=False)

,level_0,NEIGHBORHOOD_CLUSTER,CENSUS_TRACT,offensegroup,LONGITUDE,END_DATE,offense-text,SHIFT,YBLOCK,DISTRICT,...,time,uv_index,visibility,wind_bearing,wind_gust,wind_speed,error,weather_latitude,weather_longitude,crime_time
135277,276650,cluster 22,11100.0,property,-76.971563,2016-09-25T03:28:17.000,theft f/auto,midnight,139176.0,5.0,...,1.474775e+09,0.0,5.77,NaN,0.09,0.00,None,38.920452,-76.971563,2016-09-24T23:45:58.000
123130,262407,cluster 22,11100.0,property,-76.974180,2015-05-05T14:30:00.000,theft f/auto,day,139388.0,5.0,...,1.430850e+09,8.0,8.00,205.0,4.53,0.17,None,38.922362,-76.974180,2015-05-05T14:28:00.000
56648,122955,cluster 22,11100.0,violent,-76.976222,2015-07-17T22:40:00.000,assault w/dangerous weapon,evening,139347.0,5.0,...,1.437187e+09,0.0,8.44,155.0,2.53,0.67,None,38.921993,-76.976222,2015-07-17T22:35:00.000
56677,122984,cluster 22,11100.0,property,-76.974180,2015-07-18T00:45:00.000,burglary,midnight,139447.0,5.0,...,1.437181e+09,0.0,7.73,162.0,3.00,0.77,None,38.922894,-76.974180,2015-07-17T21:00:00.000
56678,122985,cluster 22,11100.0,property,-76.974180,2015-07-18T05:03:00.000,burglary,midnight,139447.0,5.0,...,1.437164e+09,4.0,8.46,177.0,5.47,1.65,None,38.922894,-76.974180,2015-07-17T16:05:00.000
56687,122994,cluster 22,11100.0,property,-76.972417,2015-07-25T01:36:00.000,theft/other,midnight,139028.0,5.0,...,1.437800e+09,0.0,9.25,NaN,0.04,0.00,None,38.919119,-76.972417,2015-07-25T01:00:00.000
174617,360844,cluster 22,11100.0,property,-76.972174,2015-12-06T03:13:33.000,theft f/auto,midnight,139441.0,5.0,...,1.449378e+09,0.0,7.08,NaN,0.02,0.00,None,38.922839,-76.972174,2015-12-06T00:00:48.000
56712,123019,cluster 24,11100.0,violent,-76.967309,2015-08-09T02:30:00.000,robbery,midnight,139007.0,5.0,...,1.439101e+09,0.0,9.13,NaN,0.03,0.00,None,38.918928,-76.967309,2015-08-09T02:09:00.000
123138,262415,cluster 22,11100.0,property,-76.974191,2015-05-10T01:40:00.000,theft f/auto,midnight,139566.0,5.0,...,1.431236e+09,0.0,7.73,172.0,1.10,0.39,None,38.923966,-76.974191,2015-05-10T01:35:00.000
56773,123080,cluster 22,11100.0,violent,-76.975116,None,sex abuse,evening,138824.0,5.0,...,1.445298e+09,0.0,7.95,180.0,0.89,0.31,None,38.917282,-76.975116,2015-10-19T19:37:53.000


Add date/time/3-hr time fields to crime/census/weather df

In [ ]:
df['START_DATE'] = pd.to_datetime(df['START_DATE'])

df['START_YEAR'] = df['START_DATE'].dt.year
df['START_MONTH'] = df['START_DATE'].dt.month
df['START_DAY'] = df['START_DATE'].dt.day

df['START_TIME_CATEGORY'] = pd.cut(df['START_DATE'].dt.hour,8,labels=['12am-3am','3am-6am','6am-9am','9am-12pm','12pm-3pm','3pm-6pm','6pm-9pm','9pm-12am'],include_lowest=True)

In [ ]:
df.head()

,level_0,NEIGHBORHOOD_CLUSTER,CENSUS_TRACT,offensegroup,LONGITUDE,END_DATE,offense-text,SHIFT,YBLOCK,DISTRICT,...,wind_gust,wind_speed,error,weather_latitude,weather_longitude,crime_time,START_YEAR,START_MONTH,START_DAY,START_TIME_CATEGORY
0,12083,cluster 26,7100.0,violent,-76.984372,2017-11-13T09:59:36.000,robbery,day,134668.0,1.0,...,1.17,0.37,None,38.879845,-76.984372,2017-11-13T08:12:36.000,2017.0,11.0,13.0,6am-9am
1,12084,cluster 27,7200.0,property,-77.004356,2017-11-13T11:52:20.000,theft f/auto,day,134352.0,1.0,...,3.42,1.09,None,38.876999,-77.004356,2017-11-13T10:26:22.000,2017.0,11.0,13.0,9am-12pm
2,12085,cluster 22,11100.0,violent,-76.974191,2017-11-13T11:35:34.000,robbery,day,139566.0,5.0,...,3.81,1.19,None,38.923966,-76.974191,2017-11-13T10:35:32.000,2017.0,11.0,13.0,9am-12pm
3,12086,cluster 24,11100.0,property,-76.965771,2017-11-17T14:26:42.000,theft/other,evening,139904.0,5.0,...,6.95,2.46,None,38.927008,-76.965771,2017-11-17T14:24:11.000,2017.0,11.0,17.0,12pm-3pm
4,12087,cluster 31,7809.0,violent,-76.927129,2017-11-17T22:57:16.000,assault w/dangerous weapon,evening,136994.0,6.0,...,0.27,0.00,None,38.900776,-76.927129,2017-11-17T20:08:53.000,2017.0,11.0,17.0,6pm-9pm


Need to update Block Group field in census/date/time df so it will join correctly to crime/weather/census

In [ ]:
full_date_time_census['Tract'] = full_date_time_census['Tract'].apply(str)

full_date_time_census['Tract'] = full_date_time_census['Tract'].apply(lambda x: x.zfill(6))

In [ ]:
full_date_time_census['BlockGroup_clean'] = full_date_time_census['Tract'] + " " + full_date_time_census['BlockGroup']

In [ ]:
full_date_time_census.head()

,date,year,month,day,time_block,Unnamed: 0,TotalPop,TPopMargin,UnWgtSampleCtPop,PerCapitaIncome,...,MedianAgeMargin,HousingUnits,HousingUnitsMargin,UnweightedSampleHousingUnits,State,County,Tract,BlockGroup,Year,BlockGroup_clean
0,2013-01-01,2013,1,1,12am-3am,433,785.0,326.0,73.0,32862.0,...,11.3,359.0,85.0,29.0,11,1,002202,3,2013,002202 3
1,2013-01-01,2013,1,1,12am-3am,434,1247.0,245.0,110.0,34004.0,...,10.2,442.0,71.0,46.0,11,1,002301,1,2013,002301 1
2,2013-01-01,2013,1,1,12am-3am,435,1621.0,232.0,153.0,34506.0,...,4.3,721.0,76.0,68.0,11,1,002301,2,2013,002301 2
3,2013-01-01,2013,1,1,12am-3am,436,1808.0,277.0,149.0,34318.0,...,20.8,1093.0,101.0,68.0,11,1,002302,1,2013,002302 1
4,2013-01-01,2013,1,1,12am-3am,437,787.0,213.0,64.0,34204.0,...,14.3,349.0,77.0,26.0,11,1,002400,1,2013,002400 1


Joining crime/weather to full date/timeblock data

In [ ]:
time_blocks_and_crime = full_date_time_census.merge(df, left_on=['year','month','day','time_block'], right_on=['START_YEAR','START_MONTH','START_DAY','START_TIME_CATEGORY'])